<a href="https://colab.research.google.com/github/ardex8752022/Distribution-of-goods-receipts-between-stores/blob/main/Project_3_Otchesova_A_V_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PROJECT-3. Решение комплексной бизнес-задач

## ЗАДАЧИ (ФОРМАЛИЗОВАННЫЕ):

    1. Построить рекомендательную систему, благодаря которой можно будет предлагать клиентам интересные им курсы. Для этого потребуется подготовить и проанализировать имеющиеся данные.
    2. Составить итоговую таблицу с рекомендациями, снабдив её необходимыми комментариями, и представить  продакт-менеджеру.
    3. Проанализировать результаты А/Б-теста, проведённого после внедрения фичи, сделать вывод.

**Для начала загрузим все необходимые библиотеки и проанализируем данные в , посмотрим, какие таблицы и с какими данными у нас есть**

In [ ]:
import pandas as pd
import numpy as np
from itertools import combinations
from collections import Counter
import psycopg2
import psycopg2.extras

/usr/local/lib/python3.7/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [ ]:
def getData():
    query = '''
select c.table_schema,
		c.table_name,
		c.column_name,
		c.data_type,
		c.ordinal_position
from information_schema.columns c
where c.table_schema = 'final'
    '''.format()
    conn = psycopg2.connect(dbname='skillfactory',
                            user='skillfactory',
                            host='84.201.134.129',
                            password='cCkxxLVrDE8EbvjueeMedPKt',
                            port=5432)
    dict_cur = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
    dict_cur.execute(query)
    rows = dict_cur.fetchall()
    data = []
    for row in rows:
        data.append(dict(row))
    dict_cur.close()
    conn.close()
    return data
df=pd.DataFrame(getData())
df

,table_schema,table_name,column_name,data_type,ordinal_position
0,final,cart_items,id,integer,1
1,final,cart_items,cart_id,integer,2
2,final,cart_items,created_at,timestamp without time zone,3
3,final,cart_items,updated_at,timestamp without time zone,4
4,final,cart_items,resource_type,character varying,5
5,final,cart_items,resource_id,integer,6
6,final,carts,id,integer,1
7,final,carts,created_at,timestamp without time zone,2
8,final,carts,updated_at,timestamp without time zone,3
9,final,carts,purchased_at,timestamp without time zone,4


**Объединим таблицы для дальнейшего анализа и проверим, за какие года есть продажи в наших данных, для этого отсортируем данные по успешными продажам курсов**

In [ ]:
def getData():
    query = '''
Select distinct(extract (year from c.purchased_at))
from final.carts c
join final.cart_items ci on c.id=ci.cart_id
where c.state='successful' and ci.resource_type='Course'
    '''.format()
    conn = psycopg2.connect(dbname='skillfactory',
                            user='skillfactory',
                            host='84.201.134.129',
                            password='cCkxxLVrDE8EbvjueeMedPKt',
                            port=5432)
    dict_cur = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
    dict_cur.execute(query)
    rows = dict_cur.fetchall()
    data = []
    for row in rows:
        data.append(dict(row))
    dict_cur.close()
    conn.close()
    return data
df=pd.DataFrame(getData())
df.head()

,date_part
0,2018.0
1,2017.0


**Сколько клиентов покупали курсы?**

In [ ]:
def getData():
    query = '''
Select count(distinct(c.user_id))
from final.carts c
join final.cart_items ci on c.id=ci.cart_id
where c.state='successful' and ci.resource_type='Course'
    '''.format()
    conn = psycopg2.connect(dbname='skillfactory',
                            user='skillfactory',
                            host='84.201.134.129',
                            password='cCkxxLVrDE8EbvjueeMedPKt',
                            port=5432)
    dict_cur = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
    dict_cur.execute(query)
    rows = dict_cur.fetchall()
    data = []
    for row in rows:
        data.append(dict(row))
    dict_cur.close()
    conn.close()
    return data
df=pd.DataFrame(getData())
df.head()

,count
0,49006


**Сколько всего есть различных курсов?**

In [ ]:
def getData():
    query = '''
Select count(distinct ci.resource_id)
from final.cart_items as ci
where ci.resource_type='Course'
    '''.format()
    conn = psycopg2.connect(dbname='skillfactory',
                            user='skillfactory',
                            host='84.201.134.129',
                            password='cCkxxLVrDE8EbvjueeMedPKt',
                            port=5432)
    dict_cur = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
    dict_cur.execute(query)
    rows = dict_cur.fetchall()
    data = []
    for row in rows:
        data.append(dict(row))
    dict_cur.close()
    conn.close()
    return data
df=pd.DataFrame(getData())
df.head()

,count
0,127


**Каково среднее число купленных курсов на одного клиента?**

In [ ]:
def getData():
    query = '''
Select round(avg(f.res),2)
from (Select count (ci.resource_id)/count(distinct c.user_id) res
from final.carts c inner join final.cart_items ci on c.id=ci.cart_id
where c.state='successful' and ci.resource_type='Course'
group by c.user_id)f
    '''.format()
    conn = psycopg2.connect(dbname='skillfactory',
                            user='skillfactory',
                            host='84.201.134.129',
                            password='cCkxxLVrDE8EbvjueeMedPKt',
                            port=5432)
    dict_cur = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
    dict_cur.execute(query)
    rows = dict_cur.fetchall()
    data = []
    for row in rows:
        data.append(dict(row))
    dict_cur.close()
    conn.close()
    return data
cart_items=pd.DataFrame(getData())
cart_items.head()

,round
0,1.44


**Сколько клиентов купили больше одного курса?**

In [ ]:
def getData():
    query = '''
Select sum(f.id)
from (select count(distinct c.user_id) id
from final.carts c inner join final.cart_items ci on c.id=ci.cart_id
where c.state='successful' and ci.resource_type='Course'
group by c.user_id
having count(distinct ci.resource_id)>1) f
    '''.format()
    conn = psycopg2.connect(dbname='skillfactory',
                            user='skillfactory',
                            host='84.201.134.129',
                            password='cCkxxLVrDE8EbvjueeMedPKt',
                            port=5432)
    dict_cur = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
    dict_cur.execute(query)
    rows = dict_cur.fetchall()
    data = []
    for row in rows:
        data.append(dict(row))
    dict_cur.close()
    conn.close()
    return data
cart_items=pd.DataFrame(getData())
cart_items.head()

,sum
0,12656


**Сформируем данные по продажам курсов в разрезе пользователей (по купившим больше одного курса)**

In [ ]:
def getData():
    query = '''
with user_tab as ( select c.user_id, count(ci.resource_id)
from final.carts c
join final.cart_items ci on c.id=ci.cart_id
where c.state='successful' and ci.resource_type='Course'
group by c.user_id
having count(distinct ci.resource_id)>1 )
select c.user_id, ci.resource_id
from final.carts c
inner join final.cart_items ci on c.id=ci.cart_id
inner join user_tab u on c.user_id=u.user_id
where c.state='successful' and ci.resource_type='Course';
    '''.format()
    conn = psycopg2.connect(dbname='skillfactory',
                            user='skillfactory',
                            host='84.201.134.129',
                            password='cCkxxLVrDE8EbvjueeMedPKt',
                            port=5432)
    dict_cur = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
    dict_cur.execute(query)
    rows = dict_cur.fetchall()
    data = []
    for row in rows:
        data.append(dict(row))
    dict_cur.close()
    conn.close()
    return data
final_df=pd.DataFrame(getData())
final_df.head()

,user_id,resource_id
0,1010882,490
1,906674,357
2,160494,507
3,749529,489
4,1010802,514


**Сохраним поученные данные в файл и далее перейдем к анализу на Python**

In [ ]:
final_df=pd.read_csv('final_df.csv')
final_df.head()

,user_id,resource_id
0,1010882,490
1,906674,357
2,160494,507
3,749529,489
4,1010802,514


**Удалим дубликаты**

In [ ]:
final_df=final_df.drop_duplicates()
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 34074 entries, 0 to 34171
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype
---  ------       --------------  -----
 0   user_id      34074 non-null  int64
 1   resource_id  34074 non-null  int64
dtypes: int64(2)
memory usage: 798.6 KB


**Произведем анализ полученых данных**

In [ ]:
#Произведём группировку, чтобы каждому пользователю соответствовал список всех купленных им курсов
df_grouped=final_df.groupby(by='user_id')['resource_id'].apply(lambda x:list(x)).reset_index()
df_grouped

,user_id,resource_id
0,51,"[516, 1099]"
1,6117,"[1125, 356, 357]"
2,10275,"[553, 1147]"
3,10457,"[361, 1138]"
4,17166,"[356, 357]"
...,...,...
12651,2179430,"[566, 750]"
12652,2186581,"[794, 1129, 864]"
12653,2187601,"[356, 912, 571, 765, 553]"
12654,2188926,"[743, 515]"


In [ ]:
#Сколько различных пар курсов встречаются вместе в покупках клиентов?
courses_pairs =[]
for element in df_grouped['resource_id']:
    for pair in combinations(sorted(list(element)),2):
        courses_pairs.append(pair)

print('Количество различных пар курсов составляет: ', len(set (courses_pairs)))

Количество различных пар курсов составляет:  3989


In [ ]:
#Заведём переменную с парой самых популярных курсов для вывода значений по умолчанию вместо Nan рекомендаций
s={k: v for k, v in popular.items() if v == max(popular_pairs1.values())}
for d in s.keys():
    print(d)

(551, 566)


**Удаляем дубликаты и объединяем пары курсв в один столбец**

In [ ]:
df_cp = pd.DataFrame(courses_pairs,  columns=['cource_1', 'course_2']).astype(str)
df_cp.drop_duplicates()
df_cp['pair'] = df_cp['cource_1'] + "," + df_cp['course_2']
df_cp.describe(include='all')
df_cp

,cource_1,course_2,pair
0,516,1099,"516,1099"
1,356,357,"356,357"
2,356,1125,"356,1125"
3,357,1125,"357,1125"
4,553,1147,"553,1147"
...,...,...,...
40012,765,912,"765,912"
40013,515,743,"515,743"
40014,756,794,"756,794"
40015,756,1185,"756,1185"


**Считаем, сколько раз встречается каждая пара**

In [ ]:
grouped_cp=df_cp.groupby(by='pair').count()
grouped_cp=grouped_cp.reset_index()
grouped_cp=grouped_cp[['pair', 'cource_1']].rename(columns={'cource_1':'count'})
grouped_cp

,pair,count
0,"1099,1100",5
1,"1099,1101",2
2,"1099,1102",2
3,"1099,1103",4
4,"1099,1125",4
...,...,...
3984,"912,1144",2
3985,"912,1156",1
3986,"912,1186",9
3987,"912,1187",1


## Теперь создаем таблицу с рекомендациями для продакт-менеджера и отдела маркетинга.

In [ ]:
#Оставим для анализа пары курсов, поввторяющиеся большее количество раз, для этого зададим 80 процентиль
x=np.percentile(grouped_cp['count'], 80)
x

11.0

In [ ]:
popular_pairs1= {k: v for k, v in sorted(popular.items(), key = lambda item: item[1], reverse = True) if v>x}
popular_pairs1

{(551, 566): 797,
 (515, 551): 417,
 (489, 551): 311,
 (523, 551): 304,
 (566, 794): 290,
 (489, 515): 286,
 (490, 566): 253,
 (490, 551): 247,
 (570, 752): 247,
 (569, 572): 216,
 (515, 523): 213,
 (553, 745): 212,
 (489, 523): 206,
 (569, 840): 204,
 (514, 551): 200,
 (516, 745): 199,
 (515, 566): 195,
 (489, 566): 188,
 (504, 572): 184,
 (572, 840): 178,
 (551, 552): 177,
 (507, 570): 172,
 (490, 809): 163,
 (489, 490): 152,
 (507, 752): 150,
 (523, 552): 144,
 (490, 515): 143,
 (551, 570): 142,
 (504, 569): 139,
 (514, 515): 139,
 (551, 745): 138,
 (514, 566): 138,
 (502, 551): 135,
 (504, 840): 135,
 (571, 1125): 122,
 (502, 566): 120,
 (523, 566): 120,
 (570, 809): 119,
 (752, 809): 115,
 (490, 523): 114,
 (357, 571): 112,
 (523, 564): 110,
 (551, 749): 109,
 (516, 553): 107,
 (551, 777): 107,
 (551, 679): 104,
 (356, 571): 103,
 (551, 564): 103,
 (515, 749): 103,
 (568, 745): 102,
 (356, 357): 100,
 (363, 511): 99,
 (551, 571): 98,
 (551, 809): 96,
 (502, 514): 95,
 (551, 794): 

Таким образом мы определили минимальный порог повторений для каждой пары для определения популярности курсов. Минимальный порог = 11 повторений, достаточный для того, чтобы считать курс популярным.

Теперь создадим список уникальных индентификаторов курсов, к которым будем добавлять рекомендации

In [ ]:
course = final_df['resource_id'].unique()
course

array([ 490,  357,  507,  489,  514,  552,  515,  523,  569,  363,  553,
        572,  502,  359,  551,  809,  516,  563,  513,  504,  509,  368,
       1144,  776,  741,  508,  777,  745,  362,  566,  356,  794,  358,
        519,  742,  679, 1103,  503,  750,  564,  361,  571,  562,  752,
        367,  756, 1104,  360,  364,  764,  670,  743,  753,  659,  366,
        568,  518, 1102,  862,  570,  791, 1116, 1101, 1100,  664,  511,
        365,  840,  517, 1141, 1128,  757, 1146, 1152,  749,  829,  744,
        909,  755,  765, 1099, 1139,  908, 1129,  814,  837,  912, 1145,
        672, 1147, 1125, 1138, 1140,  803, 1188,  810, 1161, 1184,  813,
        907,  671, 1156, 1115, 1124, 1187, 1181,  864, 1186,  863,  830,
        835,  800,  833,  866,  865,  911, 1185,  834,  836, 1182, 1200,
       1198, 1199,  902, 1160, 1201], dtype=int64)

In [ ]:
# Создадим функцию recommend, которая будет рекомендовать 1-ый и 2-ой по популярности курс к основному курсу:
def recommend(course):
    course_list = []
    for i in popular_pairs1.keys():
        if i[0] == course:
            course_list.append((i, popular_pairs1[i]))
    sorted_course_list = sorted(course_list, key = lambda x: x[1], reverse = True)
    return sorted_course_list[:2] #выводим только два значения

In [ ]:
# Создадим цикл, который для каждого основного курса извлекает 2-ое значение пары согласно уникальным значениям курсов:
#По умолчанию вместо Nan значений используем данные самых продаваемых курсов
recommended_courses1 = pd.DataFrame(columns = ['Рекомендация № 1', 'Рекомендация № 2'])
for i in course:
    if len(recommend(i)) == 2: #когда функция выдаёт 2 рекомендации
        recommended_courses1.loc[i] = [recommend(i)[0][0][1], recommend(i)[1][0][1]]
    elif len(recommend(i)) == 1: #когда функция выдаёт 1 рекомендацию  и одно значение по умолчанию вместо Nan
            recommended_courses1.loc[i] = [recommend(i)[0][0][1], d[1]]
    else:
        recommended_courses1.loc[i] = [d[0], d[1]]  #когда функция выдаёт два значения по умолчанию

In [ ]:
#Проверим получившиеся данные на наличие пустых значений
b =  recommended_courses1.isnull().sum().sum()
b
recommended_courses1

,Рекомендация № 1,Рекомендация № 2
490,566,551
357,571,1125
507,570,752
489,551,515
514,551,515
...,...,...
1198,551,566
1199,551,566
902,551,566
1160,551,566


In [ ]:
#Создадим итоговую таблицу с основным курсом и двумя рекомендациями
recommended=recommended_courses1.reset_index().rename(columns={'index':'Основной курс'})
recommended

,Основной курс,Рекомендация № 1,Рекомендация № 2
0,490,566,551
1,357,571,1125
2,507,570,752
3,489,551,515
4,514,551,515
...,...,...,...
121,1198,551,566
122,1199,551,566
123,902,551,566
124,1160,551,566


In [ ]:
#Сохраним получившийся датафрейм в Excel для отправки таблицы с рекомендациями продакт-менеджеру и в отдел маркетинга
for_printing = pd.ExcelWriter('Recomendations.xlsx')
recommended.to_excel(for_printing)
for_printing.save()